# Preliminary Items

## Imports

In [ ]:
!pip install searchtweets

In [1]:
import os
import pandas as pd

import requests
import json

In [2]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAKdIMAEAAAAAb%2Fuz8pQrMlMyb%2BP%2BPVH7AwXN7ng%3DUdK9krgu6n07e6EQHPKLkGuZmEsVUizr4btzK5IqlgSCWcT0Zq'

Enable colab pandas interactive displays

In [3]:
from google.colab import data_table
%load_ext google.colab.data_table

## Mount Google Drive 

In [4]:
currentWorkingDir = !pwd
defaultWorkingDir = "/content"

if ( currentWorkingDir[0] == defaultWorkingDir ):
  from google.colab import drive

  drive.mount('/content/drive')
      
  %cd "/content/drive/My Drive/Colab Notebooks/stock_portfolio/"
else:
  print("Currenting running app from: ")
  !pwd


Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/stock_portfolio


# Data Understanding

## Set Search URL and Query

In [5]:
search_url = "https://api.twitter.com/1.1/tweets/search/fullarchive/Production.json"

In [17]:
# Tweets extracted already
#'("$CCC.ja" OR "Carib Cement" -"JSEInvestor.com") lang:en toDate:202012310000 fromDate:201601010000'
# ("$JBG.ja" OR "JAMAICA BROILERS" -"JSEInvestor.com") lang:en until:2020-12-31 since:2016-01-01
# ("$KW.ja" OR "KINGSTON WHARVES" -"JSEInvestor.com") lang:en until:2020-12-31 since:2016-01-01

symbol = "jbg"
symbol_full_name = "Jamaica Broilers"
continue_previous_extract = False
continue_number = 0

# Query
query_params = {
                  'query'       : '("${}.ja" OR "{}" OR "{} jamaica" -"RT" -"JSEInvestor.com" -is:retweet lang:en )'.format(symbol,symbol_full_name,symbol), 
                  'maxResults'  : "500",
                  'fromDate'    : '201601010000',
                  'toDate'      : '202101010000'
                }

if (continue_previous_extract):
  query_params['next'] = ""


## Twitter API Functions

In [18]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

## Extract Tweets and save output to JSON files

In [19]:
num = 0 #count for different tweets

if continue_previous_extract:
  num = continue_number


headers = create_headers(bearer_token)
json_response = connect_to_endpoint(search_url, headers, query_params)

# If there are 500 or less total tweets from query, extract then save it to file
if not( 'next' in json_response ):
    with open('tweets_'+symbol+str(num)+'.json', 'w') as outfile:
      outfile.write(json.dumps(json_response, indent=4, sort_keys=True))

# Otherwise, do a loop to automatically save tweets
else:
  # Create a loop to automatically save 

  with open('tweets_'+symbol+str(num)+'.json', 'w') as outfile:
    outfile.write(json.dumps(json_response, indent=4, sort_keys=True))

  query_params['next'] = json_response['next']

  num += 1

  while ( 'next' in json_response ):

    json_response = connect_to_endpoint(search_url, headers, query_params)

    with open('tweets_'+symbol+str(num)+'.json', 'w') as outfile:
        outfile.write(json.dumps(json_response, indent=4, sort_keys=True))

    if 'next' in json_response:
      query_params['next'] = json_response['next']

    num += 1



200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


# Read entire JSON files to memory (List of dictionaries)

In [24]:
from glob import glob

tweet_json = []

for filename in glob('tweets_'+symbol+'*.json'):
  print(filename)
  with open(filename, 'rb') as readfile:
    tweet_json.append(json.load(readfile))


tweets_jbg0.json
tweets_jbg1.json
tweets_jbg2.json
tweets_jbg3.json
tweets_jbg4.json
tweets_jbg5.json
tweets_jbg6.json
tweets_jbg7.json
tweets_jbg8.json
tweets_jbg9.json
tweets_jbg10.json
tweets_jbg11.json
tweets_jbg12.json
tweets_jbg13.json
tweets_jbg14.json
tweets_jbg15.json
tweets_jbg16.json
tweets_jbg17.json
tweets_jbg18.json
tweets_jbg19.json
tweets_jbg20.json
tweets_jbg21.json
tweets_jbg22.json
tweets_jbg23.json
tweets_jbg24.json
tweets_jbg25.json
tweets_jbg26.json
tweets_jbg27.json
tweets_jbg28.json
tweets_jbg29.json
tweets_jbg30.json
tweets_jbg31.json
tweets_jbg32.json
tweets_jbg33.json
tweets_jbg34.json
tweets_jbg35.json
tweets_jbg36.json
tweets_jbg37.json
tweets_jbg38.json
tweets_jbg39.json
tweets_jbg40.json
tweets_jbg41.json
tweets_jbg42.json
tweets_jbg43.json
tweets_jbg44.json
tweets_jbg45.json
tweets_jbg46.json
tweets_jbg47.json
tweets_jbg48.json
tweets_jbg49.json


# Extract specific fields to memory (List of dictionaries) 

In [25]:
tweets_dict = []
single_tweet = {}

for file in tweet_json:
  for t in file['results']:
    if t['text'][0:2] != "RT" and t['retweeted'] == False:
      single_tweet['id_str'] = t['id_str']
      single_tweet['created_at'] = t['created_at']
      single_tweet['place'] = t['place']
      single_tweet['source'] = t['source']
      single_tweet['text'] = t['text']
      single_tweet['lang'] = t['lang']
      if t['truncated'] == True:
        single_tweet['text'] = t['extended_tweet']['full_text']
      single_tweet['coordinates'] = t['coordinates']
      single_tweet['geo'] = t['geo']
      single_tweet['coordinates'] = t['coordinates']
      single_tweet['is_quote_status'] = t['is_quote_status']
      single_tweet['retweet_count'] = t['retweet_count']
      single_tweet['quote_count'] = t['quote_count']
      single_tweet['reply_count'] = t['reply_count']

      single_tweet['user_'+'created_at'] = t['user']['created_at']
      single_tweet['user_'+'description'] = t['user']['description']
      single_tweet['user_'+'followers_count'] = t['user']['followers_count']
      single_tweet['user_'+'following'] = t['user']['following']
      single_tweet['user_'+'friends_count'] = t['user']['friends_count']
      single_tweet['user_'+'id_str'] = t['user']['id_str']
      single_tweet['user_'+'location'] = t['user']['location']
      single_tweet['user_'+'name'] = t['user']['name']
      single_tweet['user_'+'statuses_count'] = t['user']['statuses_count']
      single_tweet['user_'+'created_at'] = t['user']['created_at']
      single_tweet['user_'+'created_at'] = t['user']['created_at']

      tweets_dict.append(single_tweet)
      single_tweet = {}

In [26]:
tweets_dict

[{'coordinates': None,
  'created_at': 'Thu Dec 31 21:46:17 +0000 2020',
  'geo': None,
  'id_str': '1344761824051396611',
  'is_quote_status': False,
  'lang': 'pt',
  'place': None,
  'quote_count': 0,
  'reply_count': 0,
  'retweet_count': 0,
  'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  'text': '@RMartinovic jbg ja i Bole morali da duvamo protiv stresa https://t.co/gcTAToViQR',
  'user_created_at': 'Mon Jun 05 22:01:46 +0000 2017',
  'user_description': 'Respice post te! Hominem te esse memento! Memento mori!',
  'user_followers_count': 189,
  'user_following': None,
  'user_friends_count': 447,
  'user_id_str': '871849552122454018',
  'user_location': 'Republic of Serbia',
  'user_name': 'Predrag Živković',
  'user_statuses_count': 3585},
 {'coordinates': None,
  'created_at': 'Thu Dec 31 21:43:13 +0000 2020',
  'geo': None,
  'id_str': '1344761048776241155',
  'is_quote_status': False,
  'lang': 'und',
  'place': None,
  'quote_count': 0

# Create dataframe

In [13]:
df = pd.DataFrame(tweets_dict)

In [14]:
df.describe()

,retweet_count,quote_count,reply_count,user_followers_count,user_friends_count,user_statuses_count
count,51447.000000,51447.000000,51447.000000,5.144700e+04,5.144700e+04,5.144700e+04
mean,0.435225,0.051801,0.326491,1.618365e+04,9.486813e+02,7.187785e+04
std,9.245014,1.596821,1.716341,2.762348e+05,9.757324e+03,1.300630e+06
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000e+00
25%,0.000000,0.000000,0.000000,7.300000e+01,1.210000e+02,3.182000e+03
50%,0.000000,0.000000,0.000000,2.680000e+02,3.590000e+02,1.105700e+04
75%,0.000000,0.000000,0.000000,8.680000e+02,7.450000e+02,3.378400e+04
max,1819.000000,311.000000,300.000000,1.687699e+07,1.699187e+06,5.481489e+07


In [15]:
data_table.DataTable(df, include_index=False, num_rows_per_page=10,max_columns=30)

Output hidden; open in https://colab.research.google.com to view.

# Save dataframe to CSV File

In [16]:
df.to_csv("tweets_"+symbol+"_raw.csv",index=False)